# Roadmap

$$\text{Relative Response: } T \times (N * B)$$
$$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\downarrow{\frac{\Sigma{\text{ (columns: T)}}} {\text{T in E}}} $$
$$ PSTH $$
<br>
$$\text{T = trials | N = neurons | B = bins | E = events}$$

# Goal
* Create basic Euclidean distance classifier
  * Use classifier to classify tilt type based on the psths
* Use leave-one-out to create an offline confusion matrix
  * Assess performance of the trials offline
  * Extract mutual information from the confusion matrix

# Euclidean Distance
<br>
$$ distance = \sqrt{\Sigma_{b=1:totBins}(psth - \mathrm{trial\_psth})^2}$$
Dimensions:
PSTH: 1 X (tot_bins $*$ Neurons)  
Trial: 1 X (tot_bins $*$ Neurons)

$$\pagebreak$$

# PSTH CLassification Example
## Train Relative Responses
$$\text{event 1 relative response for 2 Neurons}$$
$$
\begin{bmatrix}\text{trial 1}\\\text{trial 2}\\\text{trial 3}\\\text{trial 4}\end{bmatrix}
\begin{bmatrix}0&2&4&3\\0&1&0&1\\1&0&0&1\\0&0&0&1\end{bmatrix}
\begin{bmatrix}0&2&1&3\\0&0&2&1\\0&0&1&1\\0&3&1&1\end{bmatrix}
$$
$$\text{event 1 psth template} = \begin{bmatrix}0.25&0.75&1&1.5\end{bmatrix}\begin{bmatrix}0&1.25&1.25&1.5\end{bmatrix}$$
<br>
$$\text{event 2 relative response for 2 Neurons}$$
$$
\begin{bmatrix}\text{trial 1}\\\text{trial 2}\\\text{trial 3}\end{bmatrix}
\begin{bmatrix}0&0&2&3\\1&0&1&0\\0&2&3&1\end{bmatrix}
\begin{bmatrix}1&0&2&3\\0&0&1&0\\0&0&1&3\end{bmatrix}
$$
$$\text{event 2 psth template} = \begin{bmatrix}0.33&0.67&2&1.33\end{bmatrix}\begin{bmatrix}0.33&0&1.33&2\end{bmatrix}$$

## Event 1 Template
$$\begin{bmatrix}0.25&0.75&1&1.5\end{bmatrix}\begin{bmatrix}0&1.25&1.25&1.5\end{bmatrix}$$

## Event 2 Template
$$\begin{bmatrix}0.33&0.67&2&1.33\end{bmatrix}\begin{bmatrix}0.33&0&1.33&2\end{bmatrix}$$

## Test Trial (True Label = Event 2)
$$\text{test trial} = \begin{bmatrix}0&1&2&3\end{bmatrix}\begin{bmatrix}0&0&1&2\end{bmatrix}$$

## Calculate Euclidian Distance

### Event 1 Distance
$$ \text{neuron 1} = (0.25-0)^2 + (0.75-1)^2 + (1-2)^2 + (1.5-3)^2 = \frac {27} {8}$$
$$ \text{neuron 2} = (0-0)^2 + (1.25-0)^2 + (1.25-1)^2 + (1.5-2)^2 = \frac {15} {8}$$
$$ \text{distance} = \sqrt{\frac {27} {8} + \frac{15} {8}} = \frac{\sqrt{21}} {2} = 2.291$$

### Event 2 Distance
$$ \text{neuron 1} = (0.33-0)^2 + (0.67-1)^2 + (2-2)^2 + (1.33-3)^2 = 3$$
$$ \text{neuron 2} = (0.33-0)^2 + (0-0)^2 + (1.33-1)^2 + (2-2)^2 = \frac{2} {9}$$
$$ \text{distance} = \sqrt{3 + \frac{2} {9}} = \frac{\sqrt{29}} {3} = 1.795$$

## Classifying on Min Distance
* Event 1 and test trial distance: 2.291
* Event 2 and test trial distance: 1.795
* Test trial is closest to the event 2 template, therefore that trial is classified as event 2
  * true label and predicted label match so trial was correctly identified

# Leave One Out Cross Validation
1. Remove a test trial without test trial
2. Recalculate template with trial left out
3. Classify trial
4. Replace trial removed
5. Remove next trial
6. Recreate PSTH templates without test trial
7. Calculate min distance and classify trial
8. Repeat process until every trial has been treated as a test trial

# Classic Confusion Matrix
$$\begin{bmatrix}&\text{predicted yes}&\text{predicted no}\\\text{true yes}&\text{true positive}&\text{false negative, type II error}\\\text{true no}&\text{false positive, type I error}&\text{true negative}\end{bmatrix}$$

### Accuracy (performance)
$$Accuracy = \frac{TP + TN}{\text{tot population}}$$

### Sensitivity
$$\text{True Positive Rate} = \frac{TP}{TP+FN}$$

### Specificity
$$\text{True Negative Rate} = \frac{TN}{TN+FP}$$

![Sig 007c Event 4 PSTH](images/type_1_type_2_error.jpg)
Ellis, P. D. (2016). The essential guide to effect sizes: statistical power, meta-analysis, and the interpretation of research results. Cambridge: Cambridge University Press.

# Building Classification Confusion Matrix
1. Store predicted trial and true trial from classification
2. After all trials are classified you can use the following functions

## Matlab Confusion Matrix
```m
% true_events = list of true event labels in order of classification
% predicted_events = list of predicted event labels in order of classification
result_confusion = confusionmat(true_events, predicted_events);
```

## Python Confusion Matrix
```py
from sklearn.metrics import confusion_matrix, accuracy_score
# true_events = list of true event labels in order of classification
# predicted_events = list of predicted event labels in order of classification
result_confusion = confusion_matrix(true_events, predicted_events)
```

# Classification Confusion Matrix
$$\begin{bmatrix}
&\text{predicted 1}&\text{predicted 2}&\text{predicted 3}&\text{predicted 4}\\
\text{true 1}&40&27&2&0\\
\text{true 2}&19&47&3&1\\
\text{true 3}&10&6&47&18\\
\text{true 4}&1&1&24&54
\end{bmatrix}$$

### Entropy
1. Find row entropy  
$H_\text{row} = -\Sigma_{i = 1..row}\;{p(row_i)*log_2(p(row_i))}$
2. Find column entropy  
$H_\text{column} = -\Sigma_{j = 1..column}{p(column_j)*log_2(p(column_j))}$
3. Find element entropy  
$H_\text{element} = -\Sigma_{k = 1..element}{p(element_k)*log_2(p(element_k))}$

### Information Confusion
<br>
$$I = H_a + H_b - H_{ab}$$
$$H_a = \text{row entropy} | H_b = \text{col entropy} | H_{ab} = \text{element entropy}$$

$$\pagebreak$$

# What to turn in
## Conceptual Questions (10pts)
* pdf with answers to conceptual questions
* Code files you wrote to create JSON output file
* Code files should be plain text with proper file extension
  * ex: last_first_hw3.m, last_first_hw3.py, etc.

# Json Output File (10pts)
* This file will be input into automatic grader
* This was taken from the example solution json and is a small snippet
* You are not required to turn in a pretty formatted json file
```json
{
    "bin_200ms": {
        "performance": 0.6266666666666667,
        "confusion_matrix": [
            [
                40,27,2,0
            ],
            [
                19,47,3,1
            ],
            [
                10,6,47,18
            ],
            [
                1,1,24,54
            ]
        ],
        "mutual_info": 0.7499828172100611
    }
}
```